In [2]:
'''
Main program
'''
from pathlib import Path
import os, shutil
import zipfile
import tarfile
import glob

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import xarray as xr
import rioxarray as rio
import logging

from s2driver import driver_S2_SAFE as S2
import grs
from grs import product, acutils, cams_product, l2a_product
from grs.fortran.grs import main_algo as grs_solver

opj = os.path.join
grs.__version__

'2.0.2'

In [5]:
file = '/media/harmel/vol1/Dropbox/satellite/S2/L2A/S2B_MSIL2Agrs_20220731T103629_N0400_R008_T31TFJ_20220731T124834.nc'
file = '/data/satellite/sentinel2/L2A/S2B_MSIL2Agrs_20220731T103629_N0400_R008_T31TFJ_20220731T124834.nc'
l2a = xr.open_dataset(file,decode_coords='all')

In [6]:
raster = l2a[['Rrs','Rrs_g']].rio.reproject("EPSG:3857")


In [20]:
output = l2a.Rrs.to_dataset(dim='wl')
for var in output.keys():
    output.rename({var:'Rrs_'+str(var)})
    print(output[var].name)

443
490
560
665
705
740
783
842
865
1610
2190


In [18]:
output.rename({var:'Rrs_'+str(var)})

<xarray.Dataset>
Dimensions:      (y: 5490, x: 5490)
Coordinates:
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 4.9e+06 4.9e+06 4.9e+06 ... 4.79e+06 4.79e+06
    spatial_ref  int64 ...
    time         datetime64[ns] 2022-07-31T10:36:29.024000
Data variables:
    443          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    490          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    560          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    665          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    705          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    740          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    783          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    842          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    865          (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    1610         (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    Rrs_2190     (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan


## Plot and interact


In [ ]:
variable = 'Rrs' 
raster_ = l2a[variable]#L2grs #masked[param] 
vmax = 0.03
cmap='RdBu_r'
cmap='Spectral_r'

In [ ]:
from holoviews import streams
import holoviews as hv
import panel as pn
import param
import numpy as np
import xarray as xr
hv.extension('bokeh')
from holoviews import opts


opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='binary_r', width=800, height=700),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=900),
    opts.Overlay(show_legend=True))
# set the parameter for spectra extraction
hv.extension('bokeh')
pn.extension()



third_dim = 'wl'

wl= raster_.wl.data
Nwl = len(wl)
ds = hv.Dataset(raster_.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap=cmap ,colorbar=True,clim=(0.00,vmax)).hist(bin_range=(0,0.2)) 

polys = hv.Polygons([])
box_stream = hv.streams.BoxEdit(source=polys)
dmap, dmap_std=[],[]

def roi_curves(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'Wavelength (nm)', variable)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.mean).data
        std = selection.aggregate(third_dim, np.std).data
        wl = mean.wl

        curves[i]= hv.Curve((wl,mean[variable]),'Wavelength (nm)', variable) 

    return hv.NdOverlay(curves)


# a bit dirty to have two similar function, but holoviews does not like mixing Curve and Spread for the same stream
def roi_spreads(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'Wavelength (nm)', variable)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.mean).data
        std = selection.aggregate(third_dim, np.std).data
        wl = mean.wl

        curves[i]=  hv.Spread((wl,mean[variable],std[variable]),fill_alpha=0.3)

    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMap({wl[i]: hv.VLine(wl[i]) for i in range(Nwl)},third_dim )

widget = pn.widgets.RangeSlider(start=0, end=vmax,step=0.001)

jscode = """
    color_mapper.low = cb_obj.value[0];
    color_mapper.high = cb_obj.value[1];
"""
link = widget.jslink(im, code={'value': jscode})

hv.output(widget_location='top_left')

# visualize and play
graphs = ((mean* std *hlines).relabel(variable))
layout = (im * polys +graphs    ).opts(opts.Image(tools=['hover']),
    opts.Curve(width=750,height=500, framewise=True,xlim=(400,1140),tools=['hover']), 
    opts.Polygons(fill_alpha=0.2, color='green',line_color='black'), 
    opts.VLine(color='black')).cols(2)
layout 

In [ ]:
import grstbx
from grstbx import visual

In [ ]:
import datetime as dt
time = dt.datetime.strptime(l2a.acquisition_date, '%Y-%m-%dT%H:%M:%S.%fZ')
raster = raster.expand_dims({'time':[time]})

In [ ]:
raster

In [ ]:
import holoviews as hv

hv.extension('bokeh')
v=visual.view_spectral(raster.Rrs)

In [ ]:
v.visu()


In [ ]:
v.aoi_stream.data